In [ ]:
try:
    import getpass
    import snowflake.connector as sf  
    from sqlalchemy import create_engine
    from snowflake.sqlalchemy import URL
    import pandas as pd
    import os,sys,json,gc
    import datetime,math
    from datetime import timedelta
    import statsmodels.api as sm
    from datetime import datetime
    import os

    import pickle,warnings
    import numpy as np

    warnings.filterwarnings("ignore")
except Exception as e:
    print("Error in importing Module :",e)

In [ ]:
Base_contract_product=pd.read_csv('E:/Documents/Yagna/12th/contractproduct.csv')

In [ ]:
Base_product_cat=pd.read_csv('E:/Documents/Yagna/12th/productcategories.csv')

In [ ]:
Base_contract_product.shape

In [ ]:
Base_contract_product = Base_contract_product.drop_duplicates( keep='last')

In [ ]:
Base_contract_product.loc[Base_contract_product.ContractNumber.astype(str)=='3690006027']

In [ ]:
Base_contract_product['ContractNumber'].unique().shape##81392 unique contracts

In [ ]:
Base_product_cat.shape

In [ ]:
Base_product_cat=Base_product_cat.drop_duplicates( keep='last')

In [ ]:
Base_product_cat=Base_product_cat[['Platform Group', 'Product Platform', 'Product Family', 'Part Number','List Price']]

In [ ]:
Base_product_cat_grp=Base_product_cat.groupby(by=['Part Number'])

In [ ]:
Base_product_cat_grp.count()['List Price'].max()## no duplicates 

In [ ]:
Base_contract_product['SnSSKU']=Base_contract_product['SnSSKU'].str.upper()
Base_contract_product['ProductSKU']=Base_contract_product['ProductSKU'].str.upper()
Base_product_cat['Part Number']=Base_product_cat['Part Number'].str.upper()

In [ ]:
print(Base_contract_product.columns)
(Base_product_cat.columns)

In [ ]:
Base_contract_prod_merge=Base_contract_product.merge(Base_product_cat,how='left',left_on='ProductSKU', right_on='Part Number',suffixes=('','_PRODUCT'), copy=True)

In [ ]:
Base_contract_prod_merge=Base_contract_prod_merge.merge(Base_product_cat,how='left',left_on='SnSSKU', right_on='Part Number',suffixes=('_PRODUCT','_SERVICES'), copy=True)

In [ ]:
Base_contract_prod_merge.head()

In [ ]:
Base_mismatch=Base_contract_prod_merge.loc[(Base_contract_prod_merge['Part Number_PRODUCT'].isnull()==1) & (Base_contract_prod_merge['Part Number_SERVICES'].isnull()==1) ]

In [ ]:
print(Base_contract_prod_merge.shape)
print(Base_mismatch.shape)
print(Base_mismatch.ContractNumber.unique().shape)

In [ ]:
19264*100/81392

### 23 % of our contract doesn't have have matching product or services

In [ ]:
Base_contract_prod_merge.loc[Base_contract_prod_merge['Part Number_PRODUCT'].isnull()==1,'ContractNumber'].unique().shape[0]*100/81392

### 68.41% of our contract doesn't have a matching product 

In [ ]:
Base_contract_prod_merge.loc[Base_contract_prod_merge['Part Number_SERVICES'].isnull()==1,'ContractNumber'].unique().shape[0]*100/81392

### 31.6 % of our data don't have matching services

In [ ]:
Base_contract_prod_merge.columns

In [ ]:
Base_contract_prod_merge.info()

In [ ]:
Base_contract_prod_merge['List Price_PRODUCT']=Base_contract_prod_merge['List Price_PRODUCT'].astype(float)
Base_contract_prod_merge['List Price_SERVICES']=Base_contract_prod_merge['List Price_SERVICES'].astype(float)

In [ ]:
Base_contract_prod_merge['services_price']=(Base_contract_prod_merge['List Price_SERVICES'].fillna(0))

In [ ]:
Base_contract_prod_merge['Product_price']=Base_contract_prod_merge['Quantity'].fillna(0)*Base_contract_prod_merge['List Price_PRODUCT'].fillna(0)

In [ ]:
Base_contract_prod_merge['Total_Price']=Base_contract_prod_merge['services_price'].fillna(0)+Base_contract_prod_merge['Product_price'].fillna(0)

In [ ]:
Base_contract_prod_merge=Base_contract_prod_merge.sort_values(by=['ContractNumber'])

In [ ]:
Base_contract_prod_merge

In [ ]:
Base_contract_prod_merge.ContractNumber=Base_contract_prod_merge.ContractNumber.astype(str)

In [ ]:
Base_contract_prod_merge

In [ ]:
Base_contract_prod_merge.loc[Base_contract_prod_merge.ContractNumber=='3690006027']

In [ ]:
Base_grp=Base_contract_prod_merge.groupby(['ContractNumber','OrderDate'])

In [ ]:
Base_grp_num=Base_grp.sum()

In [ ]:
Base_grp_num.reset_index(drop=False, inplace=True)

In [ ]:
Base_grp_num

In [ ]:
Base_grp_num=Base_grp_num[['ContractNumber','OrderDate','Quantity','services_price','Product_price','Total_Price']]

In [ ]:
Base_grp_ser=Base_grp['Product Platform_SERVICES'].unique()

In [ ]:
Base_grp_pro=Base_grp['Product Platform_PRODUCT'].unique()

In [ ]:
Base_grp_ser

In [ ]:
Base_grp_ser=Base_grp_ser.reset_index(drop=False)
Base_grp_pro=Base_grp_pro.reset_index(drop=False)

In [ ]:
#merge unique, order _to char trater

In [ ]:
Base_grp_ser_pro=Base_grp_ser.merge(Base_grp_pro,how='outer',on=['ContractNumber','OrderDate'])

In [ ]:
Base_grp_ser_pro.shape[0]

In [ ]:
np.concatenate((Base_grp_ser_pro.loc[5,'Product Platform_SERVICES'], Base_grp_ser_pro.loc[5,'Product Platform_PRODUCT']))

In [ ]:
Base_grp_ser_pro['Combined_pro_ser']=[ np.concatenate((Base_grp_ser_pro.loc[i,'Product Platform_SERVICES'], Base_grp_ser_pro.loc[i,'Product Platform_PRODUCT'])) for i in range(0,Base_grp_ser_pro.shape[0])]

In [ ]:
#removing  nan
Base_grp_ser_pro['Combined_pro_ser_fil']=[ list(filter(lambda v: v==v, x)) for x in Base_grp_ser_pro['Combined_pro_ser'] ]

In [ ]:
##ordering
[ x.sort() for x in Base_grp_ser_pro['Combined_pro_ser_fil'] ]

In [ ]:
#deduplicate
Base_grp_ser_pro['Combined_pro_ser_fil']=[ list(set(x))  for x in Base_grp_ser_pro['Combined_pro_ser_fil'] ]

In [ ]:
Base_grp_ser_pro

In [ ]:
#get count
Base_grp_ser_pro['Count_pro_ser']=[ len(x) for x in Base_grp_ser_pro['Combined_pro_ser_fil'] ]

In [ ]:
t=['ENT+', 'STD', 'SERVER', 'ENT+', 'STD', 'SERVER']
t.sort()
t

In [ ]:
Base_grp_ser_pro.loc[Base_grp_ser_pro.Count_pro_ser==3]

In [ ]:
##ordering
[ x.sort() for x in Base_grp_ser_pro['Combined_pro_ser_fil'] ]

In [ ]:
#creta catogoricla valriable
Base_grp_ser_pro['Cat_pro_ser']=[ '_'.join(x) for x in Base_grp_ser_pro['Combined_pro_ser_fil'] ]

In [ ]:
Base_grp_ser_pro_sel=Base_grp_ser_pro[['ContractNumber', 'OrderDate','Count_pro_ser', 'Cat_pro_ser']]

In [ ]:
pro_ser_fin=Base_grp_num.merge(Base_grp_ser_pro_sel,how='outer',on=['ContractNumber','OrderDate'])

In [ ]:
pro_ser_fin.to_csv(r'E:/Documents/Yagna/prep_data/pro_ser_fin.csv',index=False)

In [ ]:
group_presensce=pro_ser_fin.groupby('Cat_pro_ser').count().Count_pro_ser

In [ ]:
group_presensce.head()

In [ ]:
pro_ser_fin=pro_ser_fin.merge(group_presensce, how='outer',on=['Cat_pro_ser'])

In [ ]:
pro_ser_fin.columns=['ContractNumber', 'OrderDate', 'Quantity', 'services_price',
       'Product_price', 'Total_Price', 'Count_pro_ser', 'Cat_pro_ser',
       'CountForCat_pro_ser']

In [ ]:
pro_ser_fin

In [ ]:
pro_ser_fin.columns

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
sns.distplot(pro_ser_fin.Quantity.loc[pro_ser_fin.Quantity.fillna(0)<100],bins=20)

In [ ]:
pro_ser_fin['Quantity_log']=[math.log1p(i) for i in pro_ser_fin.Quantity]

In [ ]:
sns.distplot(pro_ser_fin.Quantity_log,bins=20)

In [ ]:
pro_ser_fin.columns

In [ ]:
sns.distplot(pro_ser_fin.CountForCat_pro_ser,bins=20)

In [ ]:
pro_ser_fin['CountForCat_pro_ser_bin']=pd.cut(pro_ser_fin.CountForCat_pro_ser, 20)

In [ ]:
##Categories (20, interval[float64]): [(-13.98, 750.0] < (750.0, 1499.0] < (1499.0, 2248.0] < (2248.0, 2997.0] ... (11985.0, 12734.0] < (12734.0, 13483.0] < (13483.0, 14232.0] < (14232.0, 14981.0]]

In [ ]:
pro_ser_fin

In [ ]:
sns.distplot(pro_ser_fin.Count_pro_ser,bins=20)

In [ ]:
max(pro_ser_fin.Count_pro_ser)

In [ ]:
sns.distplot(pro_ser_fin.Total_Price,bins=20)

In [ ]:
pro_ser_fin['Total_Price_log']=[math.log1p(i) for i in pro_ser_fin.Total_Price]

In [ ]:
sns.distplot(pro_ser_fin.Total_Price_log,bins=20)

In [ ]:
pro_ser_fin['Product_price_log']=[math.log1p(i) for i in pro_ser_fin.Product_price]

In [ ]:
pro_ser_fin['services_price_log']=[math.log1p(i) for i in pro_ser_fin.services_price]

In [ ]:
sns.distplot(pro_ser_fin.Product_price_log,bins=20)

In [ ]:
sns.distplot(pro_ser_fin.services_price_log,bins=20)

In [ ]:
pro_ser_fin.head()

In [ ]:
pro_ser_fin.to_csv(r'E:/Documents/Yagna/prep_data/pro_ser_fin_processed.csv',index=False)